In [ ]:
! pip install transformers


In [ ]:
! pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 848.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
csv_file_path = '/content/10000 Sales Records.csv'

table = pd.read_csv(csv_file_path)

print(table.head())


                         Region                           Country  \
0            Sub-Saharan Africa                              Chad   
1                        Europe                            Latvia   
2  Middle East and North Africa                          Pakistan   
3            Sub-Saharan Africa  Democratic Republic of the Congo   
4                        Europe                    Czech Republic   

         Item Type Sales Channel Order Priority  Order Date   Order ID  \
0  Office Supplies        Online              L   1/27/2011  292494523   
1        Beverages        Online              C  12/28/2015  361825549   
2       Vegetables       Offline              C   1/13/2011  141515767   
3        Household        Online              C   9/11/2012  500364005   
4        Beverages        Online              C  10/27/2015  127481591   

   Ship Date  Units Sold  Unit Price  Unit Cost  Total Revenue  Total Cost  \
0  2/12/2011        4484      651.21     524.96     2920025.64

In [ ]:

from transformers import TapexTokenizer, BartForConditionalGeneration

table = table.astype(str)

query = "which country has the lowest profit?"

tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

max_length = model.config.max_position_embeddings
encoding = tokenizer(table=table, query=query, return_tensors="pt", truncation=True, max_length=max_length)

outputs = model.generate(**encoding)

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


[' zambia']


In [ ]:
from transformers import TapexTokenizer, BartForConditionalGeneration

def predict_with_model(model, tokenizer, table, queries):
    predictions = []

    max_length = model.config.max_position_embeddings
    for query in queries:
        encoding = tokenizer(table=table, query=query, return_tensors="pt", truncation=True, max_length=max_length)

        outputs = model.generate(**encoding)

        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.append(prediction)

    return predictions

tokenizer = TapexTokenizer.from_pretrained("microsoft/tapex-large-finetuned-wtq")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-large-finetuned-wtq")

table = table.astype(str)

queries = []
while True:
    query = input("Enter a query (or type 'exit' to stop): ")
    if query.lower() == 'exit':
        break
    queries.append(query)

max_length = model.config.max_position_embeddings

predictions = predict_with_model(model, tokenizer, table, queries)

for query, prediction in zip(queries, predictions):
    print(f"Query: {query}")
    print(f"Prediction: {prediction}")
    print()


Enter a query (or type 'exit' to stop): which coutry lowest profit 
Enter a query (or type 'exit' to stop): which country hinghest unit prices
Enter a query (or type 'exit' to stop): exit
Query: which coutry lowest profit 
Prediction: [' russia']

Query: which country hinghest unit prices
Prediction: [' russia']

